# 导库

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision import models
from torchvision.models.resnet import resnet34
from torchvision.transforms.transforms import Resize
from tqdm import tqdm

# 数据
1. 设置数据预处理
2. 导入数据

In [53]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(),  # 随机水平镜像
    transforms.RandomErasing(scale=(0.04, 0.2), ratio=(0.5, 2)),  # 随机遮挡
    transforms.RandomCrop(224,128),  # 随机裁剪
])

In [54]:
batch_size = 128
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)  # 导入训练集
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=8) #batchsize修改了

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)  # 导入测试集
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


# 模型性能参数

In [55]:
trainAcc=[]
trainLoss=[]

# 模型
1. 导入预训练模型
2. 修改全连接层
3. 冻结其他参数
4. 损失函数 + 优化器

In [56]:
net = models.resnet34()  

In [57]:
fc_inputs = net.fc.in_features  
net.fc = nn.Sequential(  
    nn.Linear(fc_inputs, 100),  
    nn.LogSoftmax(dim=1)
)

In [58]:
layers = []
for p in net.modules():
    layers.append(p._get_name())
    if p._get_name() != 'Linear':
        p.requires_grad_ = False

with open("./layer.txt", 'w') as layer_file:
    layer_file.writelines(layers)
layers

['ResNet',
 'Conv2d',
 'BatchNorm2d',
 'ReLU',
 'MaxPool2d',
 'Sequential',
 'BasicBlock',
 'Conv2d',
 'BatchNorm2d',
 'ReLU',
 'Conv2d',
 'BatchNorm2d',
 'BasicBlock',
 'Conv2d',
 'BatchNorm2d',
 'ReLU',
 'Conv2d',
 'BatchNorm2d',
 'BasicBlock',
 'Conv2d',
 'BatchNorm2d',
 'ReLU',
 'Conv2d',
 'BatchNorm2d',
 'Sequential',
 'BasicBlock',
 'Conv2d',
 'BatchNorm2d',
 'ReLU',
 'Conv2d',
 'BatchNorm2d',
 'Sequential',
 'Conv2d',
 'BatchNorm2d',
 'BasicBlock',
 'Conv2d',
 'BatchNorm2d',
 'ReLU',
 'Conv2d',
 'BatchNorm2d',
 'BasicBlock',
 'Conv2d',
 'BatchNorm2d',
 'ReLU',
 'Conv2d',
 'BatchNorm2d',
 'BasicBlock',
 'Conv2d',
 'BatchNorm2d',
 'ReLU',
 'Conv2d',
 'BatchNorm2d',
 'Sequential',
 'BasicBlock',
 'Conv2d',
 'BatchNorm2d',
 'ReLU',
 'Conv2d',
 'BatchNorm2d',
 'Sequential',
 'Conv2d',
 'BatchNorm2d',
 'BasicBlock',
 'Conv2d',
 'BatchNorm2d',
 'ReLU',
 'Conv2d',
 'BatchNorm2d',
 'BasicBlock',
 'Conv2d',
 'BatchNorm2d',
 'ReLU',
 'Conv2d',
 'BatchNorm2d',
 'BasicBlock',
 'Conv2d',
 'Ba

In [59]:
net = net.to('cuda')

In [60]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.93)
# l = 0.05
# for i in range(100):
#     l *= 0.93
#     print(l)


In [61]:
total = 50000 # 100类 每个类500张训练 100张测试
train_loss = 0
correct = 0

# 训练

In [62]:
def train(epoch):
    total = 0
    train_loss = 0
    correct = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs = inputs.to('cuda')
        targets = targets.to('cuda')

        outputs = net(torch.squeeze(inputs, 1)) # [32, 100]

        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        print('\r', f"batch: {batch_idx+1}/{len(trainloader)} | epoch: {epoch} | Loss: {(train_loss / (batch_idx + 1))} | Accuracy: {100 * correct / total}({correct}/{total})", end='', flush=True)
    else:
        scheduler.step()
        trainAcc.append(100. * correct / total)
        trainLoss.append(train_loss / (batch_idx + 1))
        print(trainAcc, trainLoss)

In [64]:
import os
for i in range(100):
    folder = f"./model/epoch{i}"
    torch.cuda.empty_cache()
    train(i)    
    if not os.path.exists(folder):
        os.mkdir(folder)
    torch.save(net.state_dict(), f'{folder}/net{i}.pkl')
    torch.save(optimizer.state_dict(), f'{folder}/optim{i}.pkl')
    torch.save(scheduler.state_dict(), f'{folder}/sche{i}.pkl')

 batch: 391/391 | epoch: 0 | Loss: 4.161269027558739 | Accuracy: 6.408(3204/50000)0)191/49792)))[6.408] [4.161269027558739]
 batch: 391/391 | epoch: 1 | Loss: 3.8069165204187185 | Accuracy: 11.034(5517/50000)5501/49920))[6.408, 11.034] [4.161269027558739, 3.8069165204187185]
 batch: 391/391 | epoch: 2 | Loss: 3.580016196536286 | Accuracy: 14.814(7407/50000)(7395/49920))[6.408, 11.034, 14.814] [4.161269027558739, 3.8069165204187185, 3.580016196536286]
 batch: 391/391 | epoch: 3 | Loss: 3.342601071233335 | Accuracy: 18.572(9286/50000)3(9267/49920)[6.408, 11.034, 14.814, 18.572] [4.161269027558739, 3.8069165204187185, 3.580016196536286, 3.342601071233335]
 batch: 391/391 | epoch: 4 | Loss: 3.120613261561869 | Accuracy: 22.69(11345/50000)2(11324/49920)[6.408, 11.034, 14.814, 18.572, 22.69] [4.161269027558739, 3.8069165204187185, 3.580016196536286, 3.342601071233335, 3.120613261561869]
 batch: 391/391 | epoch: 5 | Loss: 2.9410779848123147 | Accuracy: 26.016(13008/50000)985/49920)))[6.408, 1

In [67]:
import os
for i in range(110, 200):
    folder = f"./model/epoch{i}"
    torch.cuda.empty_cache()
    train(i)    
    if not os.path.exists(folder):
        os.mkdir(folder)
    torch.save(net.state_dict(), f'{folder}/net{i}.pkl')
    torch.save(optimizer.state_dict(), f'{folder}/optim{i}.pkl')
    torch.save(scheduler.state_dict(), f'{folder}/sche{i}.pkl')

 batch: 391/391 | epoch: 110 | Loss: 0.573655022532129 | Accuracy: 83.064(41532/50000)41472/49920)[6.408, 11.034, 14.814, 18.572, 22.69, 26.016, 29.26, 32.266, 34.32, 36.98, 39.014, 40.638, 42.538, 43.848, 45.076, 47.096, 48.046, 48.842, 49.938, 50.984, 51.928, 52.968, 53.692, 54.51, 55.618, 56.21, 56.882, 58.076, 58.224, 59.012, 59.892, 60.094, 60.638, 61.568, 62.278, 62.402, 63.31, 63.716, 64.244, 65.082, 65.272, 65.46, 66.492, 66.634, 66.864, 67.522, 68.002, 68.082, 68.368, 69.358, 69.138, 69.992, 69.986, 70.708, 71.04, 71.274, 71.472, 72.254, 72.208, 72.522, 73.016, 73.228, 73.478, 74.04, 74.544, 74.864, 74.838, 75.152, 75.49, 75.748, 75.964, 76.03, 76.172, 76.51, 76.748, 77.038, 77.538, 77.604, 77.93, 77.804, 78.346, 78.742, 78.824, 78.9, 78.766, 79.192, 79.476, 79.706, 79.83, 80.036, 80.384, 80.568, 80.418, 80.62, 80.764, 81.076, 81.166, 81.334, 81.426, 81.524, 81.906, 82.002, 82.086, 82.08, 81.878, 82.534, 82.672, 82.408, 82.552, 82.81, 83.064] [4.161269027558739, 3.806916520418

In [ ]:
import os
for i in range(200, 400):
    folder = f"./model/epoch{i}"
    torch.cuda.empty_cache()
    train(i)    
    if not os.path.exists(folder):
        os.mkdir(folder)
    if i / 10 * 10 == i:
        torch.save(net.state_dict(), f'{folder}/net{i}.pkl')
        torch.save(optimizer.state_dict(), f'{folder}/optim{i}.pkl')
        torch.save(scheduler.state_dict(), f'{folder}/sche{i}.pkl')

In [77]:
def test():
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs = inputs.to('cuda')
            targets = targets.to('cuda')
            outputs = net(torch.squeeze(inputs, 1))
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(batch_idx, '/', len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                  % (test_loss / (batch_idx + 1), 100. * correct / total, correct, total))

In [ ]:
test()